In [1]:
from dotenv import load_dotenv
import os

In [2]:
import minsearch

In [3]:
import requests 

In [36]:
from google import genai

In [5]:
load_dotenv() 
API_KEY = os.getenv("GEMINI_API_KEY")

In [6]:
#os.environ

In [7]:
docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [8]:
#documents[0]

In [9]:
index = minsearch.Index(
    text_fields = ["question","text","section"],
    keyword_fields = ["course"]
)

In [10]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5
    )

    return results

In [11]:
pip install elasticsearch

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [31]:
from elasticsearch import Elasticsearch

In [32]:
index.fit(documents)

In [33]:
es_client = Elasticsearch('http://localhost:9200')

In [ ]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"

es_client.indices.create(index=index_name, body=index_settings)

In [ ]:
#documents[0]

In [ ]:
es_client.info()

In [ ]:
from tqdm.auto import tqdm

In [ ]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

In [16]:
def elastic_search(query):
    search_query = {
        "size": 3,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "machine-learning-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = [] 
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    
    return result_docs

In [17]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()
    
    context = ""
    
    for doc in search_results:
        context += f"question: {doc['question']}\n\nanswer: {doc['text']}\n\n"

    prompt = prompt_template.format(question=query, context=context.strip())
    return prompt


In [18]:
def llm(prompt):
    client = genai.Client(api_key=API_KEY)
    responses = client.models.generate_content(
        model="gemini-2.0-flash", contents=prompt)
    return responses

In [29]:
def rag(query):
    search_results = elastic_search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer.text

In [28]:
query = "How do I execute a command in a running docker container?"

In [37]:
result = rag(query)
print(result)

If the container is already running, find the container-id using `docker ps` and then execute a command in the specific container using `docker exec -it <container-id> bash`.



In [ ]:
#pip install -q -U google-genai

In [44]:
model = genai.GenerativeModel(model_name="gemini-2.0-flash")
search_results = elastic_search(query)
prompt = build_prompt(query, search_results)
response = model.generate_content(prompt)
token_count = model.count_tokens(prompt)
print("Token count:", token_count.total_tokens)

Token count: 327
